In [2]:
from os.path import join, basename, abspath, exists
from os import listdir, makedirs

from keras import backend as K
from keras.models import load_model

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import cv2

import time
import pickle

from jaad_data import JAAD

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
jaad_path = "/home/kara9147/jaad2"
annotation_path = join(jaad_path, 'annotations')
imdb = JAAD(data_path = jaad_path)

In [12]:
vid = "video_0318"
anno = imdb._get_annotations(vid)

bb_cross={}
ped_b = [k for k,v in anno["ped_annotations"].items() if k.endswith('b')]

for i, p in enumerate(ped_b):
    #if (1 in anno["ped_annotations"][p]["behavior"]["cross"]):
        bb_cross[p] = (anno["ped_annotations"][p]["bbox"])
        print(p)
print(ped_b)
#for k,v in bb_cross.items():
#    print(k)

ped_index = 0
lstm_seq =  bb_cross[ped_b[ped_index]]
frames = (anno['ped_annotations'][ped_b[ped_index]]['frames'][0],
          len(anno['ped_annotations'][ped_b[ped_index]]['frames']))

df = pd.DataFrame(lstm_seq)
df = df.astype('float32')

0_318_2542b
['0_318_2542b']


In [13]:
len(lstm_seq)

240

In [14]:
lstm_seq[0]

[645.0, 610.0, 784.0, 897.0]

In [15]:
pickle_in = open("min-max-scaler.pkl","rb")
scaler = pickle.load(pickle_in)
#print(type(scaler))
#print(scaler.data_max_)
#print(scaler.data_min_)

lstm_seq = scaler.transform(lstm_seq)

In [16]:
lstm_seq.shape

(240, 4)

In [17]:
model_path = "conf3-lstm-model.h5"
model = load_model(model_path)
n_seq = 30

In [36]:
i = 0

while( i < lstm_seq.shape[0] - n_seq - 1):
    print(i)
    print(df[i+30:i+31])
    i = i + 1
df[i+30:i+31]

0
        0      1      2      3
30  666.0  596.0  827.0  928.0
1
        0      1      2      3
31  665.0  598.0  826.0  930.0
2
        0      1      2      3
32  665.0  597.0  826.0  929.0
3
        0      1      2      3
33  665.0  596.0  826.0  928.0
4
        0      1      2      3
34  665.0  596.0  827.0  928.0
5
        0      1      2      3
35  665.0  595.0  827.0  927.0
6
        0      1      2      3
36  665.0  594.0  827.0  926.0
7
        0      1      2      3
37  665.0  593.0  827.0  925.0
8
        0      1      2      3
38  665.0  591.0  828.0  927.0
9
        0      1      2      3
39  664.0  589.0  828.0  928.0
10
        0      1      2      3
40  664.0  587.0  830.0  930.0
11
        0      1      2      3
41  664.0  584.0  831.0  931.0
12
        0      1      2      3
42  663.0  582.0  831.0  932.0
13
        0      1      2      3
43  663.0  580.0  832.0  934.0
14
        0      1      2      3
44  662.0  583.0  833.0  938.0
15
        0      1      2      3
4

         0      1      2       3
161  723.0  562.0  960.0  1021.0
132
         0      1      2       3
162  723.0  562.0  960.0  1021.0
133
         0      1      2       3
163  723.0  562.0  960.0  1020.0
134
         0      1      2       3
164  723.0  561.0  960.0  1019.0
135
         0      1      2       3
165  723.0  561.0  960.0  1018.0
136
         0      1      2       3
166  723.0  560.0  960.0  1017.0
137
         0      1      2       3
167  723.0  560.0  960.0  1022.0
138
         0      1      2       3
168  723.0  560.0  960.0  1029.0
139
         0      1      2       3
169  724.0  559.0  961.0  1026.0
140
         0      1      2       3
170  724.0  559.0  961.0  1027.0
141
         0      1      2       3
171  723.0  564.0  961.0  1030.0
142
         0      1      2       3
172  723.0  568.0  961.0  1032.0
143
         0      1      2       3
173  722.0  572.0  961.0  1034.0
144
         0      1      2       3
174  722.0  574.0  962.0  1035.0
145
         0      1   

,0,1,2,3
239,582.0,597.0,803.0,1027.0


In [39]:
i = 0
total_time = 0.0
pred = True
total_positive = 0

while( i < lstm_seq.shape[0] - n_seq -1):
    batch = lstm_seq[i:i + n_seq]
    i = i + 1

    current = time.time()
    ##################################PREDICTION######################
    y_hat = model.predict(np.expand_dims(batch, axis=0))
    #print(y_hat)
    #print(len(batch))

    end = time.time()
    diff = end - current
    total_time  = total_time  + diff
    #print(end - current)
    #print("Time spent for predicting: {0}".format(diff))

    # Decode the raw prediction `y_pred`
    inv_yhat = scaler.inverse_transform(y_hat)[0]
    #print(inv_yhat)

    xmin_ = int (df[i:i+1][0] / 2)
    ymin_ = int (df[i:i+1][1] / 2)
    xmax_ = int (df[i:i+1][2] / 2)
    ymax_ = int (df[i:i+1][3] / 2)

    # Draw the present boxes in green
    boxA =  df[i+30:i+31].values[0]
    boxB = inv_yhat
    
    #print(boxA)
    #print(boxB)
    
    score = bb_intersection_over_union(boxA, boxB)
    if score > 0.5 :
        total_positive += 1
print ("total #prediction: %d" % i)
print ("total time: %f" % total_time)
print ("total positive: %d" % total_positive)
print ("accuracy: %d" % (total_positive / i))


total #prediction: 209
total time: 3.365747
total positive: 209


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [44]:
print ("accuracy: %d" % (total_positive / i))

accuracy: 1


In [1]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
 
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
 
	# return the intersection over union value
	return iou